In [2]:
import pandas as pd 
import numpy as np
import sklearn

In [3]:
import sklearn
print(sklearn.__version__)

1.0.1


In [4]:
import time
def timing(f):
    def wrap(*args, **kwargs):
        time1 = time.time()
        ret = f(*args, **kwargs)
        time2 = time.time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

# Load data and x/y

In [5]:
train = pd.read_csv("data/preparedData_NSL_KDD_train_cat.csv",nrows=1000000)
test = pd.read_csv("data/preparedData_NSL_KDD_test_cat.csv",nrows=1000000)

/opt/lhm/tljh/user/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
test.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,1,2,prot_1,prot_2,prot_3,land,logged_in,is_host_login,is_guest_login,type
0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,neptune
1,0.000035,2.066513e-04,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,normal
2,0.000000,3.183413e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,saint
3,0.000017,0.000000e+00,0.000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,mscan
4,0.000000,4.249857e-06,0.010784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,normal


In [7]:
train.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,1,2,prot_1,prot_2,prot_3,land,logged_in,is_host_login,is_guest_login,type
0,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,normal
1,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,neptune
2,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,normal
3,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,normal
4,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,neptune


In [8]:
train.shape

(125972, 56)

In [9]:
test.shape

(125947, 56)

In [10]:
train = train.dropna()
test = test.dropna()

In [11]:
test = test.drop_duplicates()
train = train.drop_duplicates()

In [12]:
train.shape

(123753, 56)

In [13]:
test.shape

(22376, 56)

In [14]:
train["type"].value_counts()

normal             66057
neptune            40315
satan               3617
ipsweep             3594
portsweep           2925
smurf               2642
nmap                1490
back                 956
teardrop             891
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: type, dtype: int64

In [15]:
test["type"].value_counts()

normal             9711
neptune            4495
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               731
processtable        685
smurf               665
back                359
snmpguess           331
saint               318
mailbomb            293
snmpgetattack       178
portsweep           156
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
worm                  2
loadmodule            2
perl                  2
sqlattack             2
udpstorm              2
phf                   2
imap                  1
Name: type, dtype: int64

In [16]:
y_test = test["type"].apply(lambda x: x == "normal")
y_train = train["type"].apply(lambda x: x == "normal")

In [17]:
X_train = train.drop("type",axis = 1)
X_test =  test.drop("type",axis = 1)

In [18]:
X_train.isnull().sum().sum()

0

In [19]:
X_train.shape

(123753, 55)

In [20]:
print(str(sum(y_test)) + " / " + str(len(y_test)))

9711 / 22376


In [21]:
print(str(sum(y_train)) + " / " + str(len(y_train)))

66057 / 123753


## train

In [22]:
@timing
def train_model(model,X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [23]:
@timing
def run_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return y_pred   

In [24]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
def eval_model(y_test,y_pred):
    pres = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return pres,rec, roc, f1

In [39]:
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(learning_rate = 0.08, max_depth = 50, max_iter = 300, max_leaf_nodes =  127, min_samples_leaf = 50)
model = train_model(model,X_train,y_train)

train_model function took 3931.576 ms


In [47]:
y_pred = np.ones(len(X_test))

In [33]:
y_pred = y_pred.reshape(len(y_pred), 1)

In [44]:
sum(y_pred)

9969

In [30]:
y_pred = np.array(y_pred, dtype=bool)

In [36]:
y_pred[0] = True

In [40]:
y_pred = run_model(model, X_test, y_test)

run_model function took 37.188 ms


In [48]:
eval_model(y_test,y_pred)

(0.4339917769038255, 1.0, 0.5, 0.6052918627481534)

In [27]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(n_estimators=150, random_state=0,n_jobs = -1)
model = train_model(model,X_train,y_train)

train_model function took 16544.221 ms


In [28]:
y_pred = run_model(model, X_test, y_test)

run_model function took 858.219 ms


In [29]:
eval_model(y_test,y_pred)

(0.6722900215362527,
 0.7714962413757595,
 0.7415712553108564,
 0.7184847758331336)

In [30]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 150, random_state=0,n_jobs = -1)
model = train_model(model,X_train,y_train)

train_model function took 2843.699 ms


In [31]:
y_pred = run_model(model, X_test, y_test)

run_model function took 225.474 ms


In [32]:
eval_model(y_test,y_pred)

(0.6801399502808213,
 0.7606837606837606,
 0.7431922553912289,
 0.7181606066498153)

In [33]:
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model = train_model(model,X_train,y_train)

train_model function took 2940.314 ms


In [34]:
y_pred = run_model(model, X_test, y_test)

run_model function took 11.817 ms


In [35]:
eval_model(y_test,y_pred)

(0.7333458259158625,
 0.8059932035835651,
 0.7906397127274318,
 0.7679552590266877)

In [41]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(random_state=1, max_iter=200)
model = train_model(model,X_train.head(1000000),y_train.head(1000000))

train_model function took 146024.613 ms


In [42]:
y_pred = run_model(model, X_test, y_test)

run_model function took 107.505 ms


In [43]:
eval_model(y_test,y_pred)

(0.7204385277995301,
 0.7579034085058182,
 0.7662000264005602,
 0.7386962412806743)

In [38]:
from sklearn.svm import SVC
model = SVC(random_state=1, max_iter=200)
model = train_model(model,X_train.head(1000000),y_train.head(1000000))

train_model function took 9034.120 ms


/opt/lhm/tljh/user/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


In [39]:
y_pred = run_model(model, X_test, y_test)

run_model function took 1338.176 ms


In [40]:
eval_model(y_test,y_pred)

(0.248, 0.0031922562043043973, 0.497885113494967, 0.0063033753558357055)

In [51]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
estimators=[("gard", HistGradientBoostingClassifier(learning_rate = 0.08, max_depth = 50, max_iter = 300, max_leaf_nodes =  127, min_samples_leaf = 50)
), ("mlp",MLPClassifier(random_state=1, max_iter=200)),("SVC",XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))]
model = VotingClassifier(estimators, voting="hard")
model = train_model(model,X_train.head(1500000),y_train.head(1500000))

train_model function took 163792.726 ms


In [52]:
y_pred = run_model(model, X_test, y_test)

run_model function took 319.388 ms


In [53]:
eval_model(y_test,y_pred)

(0.737063778580024, 0.7568736484399134, 0.7749232040067708, 0.7468373723517756)